In [1]:
# https://towardsdatascience.com/how-to-program-umap-from-scratch-e6eff67f55fe

In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import euclidean_distances
from scipy import optimize

expr = pd.read_csv('static/CAFs.txt', sep='\t')

In [3]:
X_train = expr.values[:, :-1]
X_train = np.log(X_train+1)
n = X_train.shape[0]
print("This dataset contains {0} samples".format(n))
Y_train = expr.values[:, -1:]
print("\nDimensions of the data points: {0}, labels: {1}".format(X_train.shape, Y_train.shape))

This dataset contains 716 samples

Dimensions of the data points: (716, 557), labels: (716, 1)


In [4]:
# Matrix of the squared Euclidean distances
dist = np.square(euclidean_distances(X_train, X_train))

In [5]:
#Get the squared Euclidean distance to the nearest neighbor
rho = [sorted(dist[i])[1] for i in range(dist.shape[0])]

In [6]:
print(dist[0:4, 0:4])
print()
print(rho[0:4])

[[   0.          914.95016311 1477.46836099 3036.91172176]
 [ 914.95016311    0.         1307.39294642 2960.41559961]
 [1477.46836099 1307.39294642    0.         2678.34442573]
 [3036.91172176 2960.41559961 2678.34442573    0.        ]]

[805.2464562222542, 652.4022952321459, 1036.9011547563534, 1244.8783774968015]


In [7]:
def prob_high_dim(sigma, row_index):
    """
    For each row of Euclidean distance matrix (dist_row) compute
    probability in high dimensions (1D array)
    """
    d = dist[row_index] - rho[row_index]; d[d < 0] = 0
    return np.exp(-d/sigma)

In [8]:
def k(prob):
    """
    Compute n_neighbor = k (scalar) for each 1D array of high-dimensional probability
    """
    return np.power(2, np.sum(prob))

In [9]:
def sigma_binary_search(k_of_sigma, fixed_k):
    """
    Solve equation k_of_sigma(sigma) = fixed_k 
    with respect to sigma by the binary search algorithm
    """
    sigma_lower_limit = 0; sigma_upper_limit = 1000
    for i in range(20):
        approx_sigma = (sigma_lower_limit + sigma_upper_limit) / 2
        if k_of_sigma(approx_sigma) < fixed_k:
            sigma_lower_limit = approx_sigma
        else:
            sigma_upper_limit = approx_sigma
        if np.abs(fixed_k - k_of_sigma(approx_sigma)) <= 1e-5:
            break
    return approx_sigma

N_NEIGHBOR = 15
prob = np.zeros((n,n)); sigma_array = []
for row_index in range(n):
    func = lambda sigma: k(prob_high_dim(sigma, row_index))
    binary_search_result = sigma_binary_search(func, N_NEIGHBOR)
    prob[row_index] = prob_high_dim(binary_search_result, row_index)
    sigma_array.append(binary_search_result)
    if (row_index + 1) % 100 == 0:
        print("Sigma binary search finished {0} of {1} cells".format(row_index + 1, n))
print("\nMean sigma = " + str(np.mean(sigma_array)))

Sigma binary search finished 100 of 716 cells
Sigma binary search finished 200 of 716 cells
Sigma binary search finished 300 of 716 cells
Sigma binary search finished 400 of 716 cells
Sigma binary search finished 500 of 716 cells
Sigma binary search finished 600 of 716 cells
Sigma binary search finished 700 of 716 cells

Mean sigma = 63.51506110676174


In [12]:
MIN_DIST = 0.25

x = np.linspace(0, 3, 300)

def f(x, min_dist):
    y = []
    for i in range(len(x)):
        if(x[i] <= min_dist):
            y.append(1)
        else:
            y.append(np.exp(- x[i] + min_dist))
    return y

dist_low_dim = lambda x, a, b: 1/(1 + a*x**(2*b))

p, _ = optimize.curve_fit(dist_low_dim, x, f(x, MIN_DIST))

a = p[0]
b = p[1]

print("Hyperparameters a = " + str(a) + " and b = " + str(b))

Hyperparameters a = 1.1214363425627392 and b = 1.0574998764478827


In [16]:
def prob_low_dim(Y):
    """
    Compute matrix of probabilities q_ij in low_dimensional space
    """
    inv_distances = np.power(1 + a * np.square(euclidean_distances(Y,Y))**b, -1)
    return inv_distances

In [17]:
def CE(P, Y):
    """
    Compute Cross-Entropy (CE) from matrix of high-dimensional probabilities
    and coordinates of low-dimensional embeddings
    """
    Q = prob_low_dim(Y)
    return - P * np.log(Q + 0.01) - (1 - P) * np.log(1 - Q + 0.01)

def CE_gradient(P, Y):
    """
    Compute the gradient of Cross-Entropy (CE)
    """
    y_diff = np.expand_dims(Y, 1) - np.expand_dims(Y, 0)
    inv_dist = np.power(1 + a * np.square(euclidean_distances(Y, Y)) ** b, -1)
    Q = np.dot(1 - P, np.power(0.001 + np.square(euclidean_distances(Y, Y)), -1))
    np.fill_diagonal(Q, 0)
    Q = Q / np.sum(Q, axis = 1, keepdims = True)
    fact = np.expand_dims(a*P*(1e-8 + np.square(euclidean_distances(Y, Y)))**(b-1) - Q, 2)
    return 2 * b * np.sum(fact * y_diff * np.expand_dims(inv_dist, 2), axis = 1)